Get the Isomeric Smiles from a list of CAS-ID's

In [25]:
import pandas as pd
from selenium import webdriver
import time
import cirpy

In [26]:
df = pd.read_excel('Stock list Carbohydrates.xlsx', dtype=str)
df = df.fillna('')    
print(df.shape)
print(df.columns)
info = [(j,k,l) for i,j,k,l in zip(df['Storage Location US'],df['Material Code'],df['Name'],df['CAS']) if i == 'Yes' and str(l) != '']
print(info[:10])

(3021, 8)
Index(['Material Code', 'Name', 'CAS', 'Storage Location Switzerland',
       'Storage Location US', 'Storage Location Slovakia',
       'Storage Location UK', 'Storage Location China'],
      dtype='object')
[('DC06148', '7-Cyclohexylheptyl b-D-maltoside', '349477-49-2'), ('DD06199', 'n-Dodecyl-beta-D-maltoside', '69227-93-6'), ('DD06199', 'Dodecyl b-D-maltopyranoside', '69227-93-6'), ('DD06359', 'n-Dodecyl-beta-D-glucopyranoside', '59122-55-3'), ('DD06375', 'Decyl a-D-maltopyranoside', '168037-12-5'), ('DD06810', 'Dodecanoyl D-sucrose', '25339-99-5'), ('DD14033', 'Decyl maltose neopentyl glycol', '1257852-99-5'), ('DL14035', 'Lauryl maltose neopentyl glycol', '1257852-96-2'), ('DO05161', 'Octyl b-D-glucopyranoside', '29836-26-8'), ('DO06354', 'Octyl b-D-thioglucopyranoside', '85618-21-9')]


In [27]:
driver = webdriver.Firefox()

url = f'https://pubchem.ncbi.nlm.nih.gov/#query='
driver.get(url)
time.sleep(10)


isomericSMILES = []

for i,j,k in info:
    url = f'https://pubchem.ncbi.nlm.nih.gov/#query={k}'
    driver.get(url)
    print(k)
    for _ in range(10):
        time.sleep(1)
        html = driver.page_source
        x = html[html.find('Isomeric SMILES'):]
        x = x[:x.find('&nbsp;&nbsp')]
        x = x[x.find('breakword'):]
        y = x[x.find('<span>')+6:x.find('</span>')]
        if y != '':
            print(k,y)
            isomericSMILES += [[y,i,j,k]]
            break

df2 = pd.DataFrame(isomericSMILES,columns=['SMILES','Material Code','Name','CAS'])
df2.to_csv('StockListCarbohydratesFiltered.csv')
print(df2)

driver.close()

349477-49-2
349477-49-2 C1CCC(CC1)CCCCCCCO[C@H]2[C@@H]([C@H]([C@@H]([C@H](O2)CO)O[C@@H]3[C@@H]([C@H]([C@@H]([C@H](O3)CO)O)O)O)O)O
69227-93-6
69227-93-6 CCCCCCCCCCCCO[C@H]1[C@@H]([C@H]([C@@H]([C@H](O1)CO)O[C@@H]2[C@@H]([C@H]([C@@H]([C@H](O2)CO)O)O)O)O)O
69227-93-6
69227-93-6 CCCCCCCCCCCCO[C@H]1[C@@H]([C@H]([C@@H]([C@H](O1)CO)O[C@@H]2[C@@H]([C@H]([C@@H]([C@H](O2)CO)O)O)O)O)O
59122-55-3
59122-55-3 CCCCCCCCCCCCO[C@H]1[C@@H]([C@H]([C@@H]([C@H](O1)CO)O)O)O
168037-12-5
168037-12-5 CCCCCCCCCCO[C@@H]1[C@@H]([C@H]([C@@H]([C@H](O1)CO)O[C@@H]2[C@@H]([C@H]([C@@H]([C@H](O2)CO)O)O)O)O)O
25339-99-5
1257852-99-5
1257852-99-5 CCCCCCCCC(CCCCCCCC)(CO[C@H]1[C@@H]([C@H]([C@@H]([C@H](O1)CO)O[C@@H]2[C@@H]([C@H]([C@@H]([C@H](O2)CO)O)O)O)O)O)CO[C@H]3[C@@H]([C@H]([C@@H]([C@H](O3)CO)O[C@@H]4[C@@H]([C@H]([C@@H]([C@H](O4)CO)O)O)O)O)O
1257852-96-2
1257852-96-2 CCCCCCCCCCC(CCCCCCCCCC)(CO[C@H]1[C@@H]([C@H]([C@@H]([C@H](O1)CO)O[C@@H]2[C@@H]([C@H]([C@@H]([C@H](O2)CO)O)O)O)O)O)CO[C@H]3[C@@H]([C@H]([C@@H]([C@H](O3)CO)O[C@

In [28]:
df = pd.read_excel('Stock list Carbohydrates.xlsx')
df = df[df['Storage Location US'] == 'Yes']
df.reset_index(inplace = True)
casses = df['CAS']
names = df['Name']
codes = df['Material Code']

test_list = []
for x,y,z in zip(casses,names,codes):
    test_list.append([cirpy.resolve(str(x), 'smiles', ['cas_number']),y,x,z])
test_list = pd.DataFrame([i for i in test_list if i[0]], columns =['SMILES','Name','cas accession','material code'])
test_list.to_csv('StockListCarbohydratesCirpy.csv')
print(test_list)
print(test_list.shape)

                                                SMILES  \
0    CCCCCCCCCCCCO[C@@H]1O[C@H](CO)[C@@H](O[C@H]2O[...   
1    CCCCCCCCCCCCO[C@@H]1O[C@H](CO)[C@@H](O[C@H]2O[...   
2    CCCCCCCCCCCCO[C@@H]1O[C@H](CO)[C@@H](O)[C@H](O...   
3    CCCCCCCCCCCC(O)=O.OC[C@H]1O[C@H](O[C@]2(CO)O[C...   
4    CCCCCCCCO[C@@H]1O[C@H](CO)[C@@H](O)[C@H](O)[C@...   
..                                                 ...   
384  COC(=O)N[C@@H]1[C@@H](O)[C@H](O[C@@H]2O[C@H](C...   
385  OC[C@H]1O[C@@H](O[C@@H]2[C@H](O)[C@@H](O[S]([O...   
386  COC(=O)N[C@@H]1[C@@H](O)[C@H](O[C@@H]2O[C@H](C...   
387  OC[C@H]1O[C@H](OC[C@H]2O[C@H](O[C@H]3[C@H](O)[...   
388  [Na+].CC(=O)N[C@H]1[C@H](O)O[C@H](CO)[C@@H](O)...   

                                                  Name cas accession  \
0                           n-Dodecyl-beta-D-maltoside    69227-93-6   
1                          Dodecyl b-D-maltopyranoside    69227-93-6   
2                     n-Dodecyl-beta-D-glucopyranoside    59122-55-3   
3              